In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# I want to load a csv placed in ../logs/log_ear_mar.csv
# with this header timestamp;path_img;ear;mar

# And load into a dataframe using pandas


2023-09-19 19:43:26.908354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file_path = "../logs/log_all.csv"

df = pd.read_csv(file_path, sep=";", header=None, names=["timestamp", "path_img", "ear", "mar", "landmarks", "prediction"])

In [3]:
# cast ear and mar to float
df['ear'] = df['ear'].astype(float)
df['mar'] = df['mar'].astype(float)

# delete rows with ear or mar <= 0
df = df[df['ear'] > 0]
df = df[df['mar'] > 0]

In [4]:
FRAMES = 10

# add columns with min and max values for the past 100 frames of each timestamp and for category
df['ear_min'] = df['ear'].rolling(FRAMES).min()
df['ear_max'] = df['ear'].rolling(FRAMES).max()
df['mar_min'] = df['mar'].rolling(FRAMES).min()
df['mar_max'] = df['mar'].rolling(FRAMES).max()
# add columns with the mean of the past FRAMES seconds of each timestamp and for category
df['ear_mean'] = df['ear'].rolling(FRAMES).mean()
df['mar_mean'] = df['mar'].rolling(FRAMES).mean()

# add columns with the std of the past FRAMES seconds of each timestamp and for category
df['ear_std'] = df['ear'].rolling(FRAMES).std()
df['mar_std'] = df['mar'].rolling(FRAMES).std()

# add columns with the median of the past FRAMES seconds of each timestamp and for category
df['ear_median'] = df['ear'].rolling(FRAMES).median()
df['mar_median'] = df['mar'].rolling(FRAMES).median()

# add columns with the variance of the past FRAMES seconds of each timestamp and for category
df['ear_var'] = df['ear'].rolling(FRAMES).var()
df['mar_var'] = df['mar'].rolling(FRAMES).var()

# # add columns with the skew of the past FRAMES seconds of each timestamp and for category
df['ear_skew'] = df['ear'].rolling(FRAMES).skew()
df['mar_skew'] = df['mar'].rolling(FRAMES).skew()

# # add columns with the kurt of the past FRAMES seconds of each timestamp and for category
df['ear_kurt'] = df['ear'].rolling(FRAMES).kurt()
df['mar_kurt'] = df['mar'].rolling(FRAMES).kurt()

# Drop rows with NaN values
df = df.dropna()


df

,timestamp,path_img,ear,mar,landmarks,prediction,ear_min,ear_max,mar_min,mar_max,...,ear_std,mar_std,ear_median,mar_median,ear_var,mar_var,ear_skew,mar_skew,ear_kurt,mar_kurt
9,20230917213349627777,./frames/train/frame_20230917213349627777.jpg,0.303363,0.050924,"[array([479, 354]), array([485, 393]), array([...",0,0.186867,0.334583,0.040725,0.093008,...,0.047551,0.017132,0.278485,0.055068,0.002261,0.000294,-0.351794,1.097425,-0.873910,0.545117
10,20230917213349791915,./frames/train/frame_20230917213349791915.jpg,0.306768,0.065031,"[array([498, 347]), array([505, 385]), array([...",0,0.186867,0.334583,0.040725,0.084471,...,0.045256,0.012753,0.283943,0.055068,0.002048,0.000163,-0.870369,0.961032,0.270700,1.763991
11,20230917213349957318,./frames/train/frame_20230917213349957318.jpg,0.218554,0.058120,"[array([497, 356]), array([504, 393]), array([...",0,0.207164,0.334583,0.040725,0.084471,...,0.039341,0.012701,0.283943,0.057574,0.001548,0.000161,-0.563064,0.828058,-0.131470,1.641230
12,20230917213350126148,./frames/train/frame_20230917213350126148.jpg,0.278853,0.133038,"[array([497, 353]), array([504, 390]), array([...",0,0.218554,0.334583,0.040775,0.133038,...,0.031232,0.026041,0.283943,0.058472,0.000975,0.000678,-0.557587,2.187100,1.474192,5.381919
13,20230917213350290753,./frames/train/frame_20230917213350290753.jpg,0.301346,0.154350,"[array([498, 351]), array([503, 389]), array([...",0,0.218554,0.334583,0.040775,0.154350,...,0.029919,0.037408,0.286336,0.060916,0.000895,0.001399,-1.078274,1.500414,3.184245,1.154162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,20230917213445916400,./frames/train/frame_20230917213445916400.jpg,0.318163,0.038219,"[array([487, 377]), array([494, 412]), array([...",0,0.170084,0.343283,0.022974,0.223203,...,0.063079,0.059705,0.301044,0.038001,0.003979,0.003565,-0.781867,3.079077,-1.108904,9.617017
329,20230917213446071755,./frames/train/frame_20230917213446071755.jpg,0.324961,0.033878,"[array([487, 375]), array([493, 410]), array([...",0,0.170084,0.343283,0.022974,0.048956,...,0.060312,0.007106,0.308854,0.037784,0.003638,0.000050,-1.367431,0.015203,0.579666,0.849828
330,20230917213446244030,./frames/train/frame_20230917213446244030.jpg,0.325135,0.045452,"[array([488, 376]), array([493, 411]), array([...",0,0.178919,0.343283,0.029304,0.048956,...,0.046430,0.006183,0.311323,0.038001,0.002156,0.000038,-2.205985,0.331874,5.507519,0.069262
331,20230917213446410906,./frames/train/frame_20230917213446410906.jpg,0.233033,0.033170,"[array([488, 375]), array([494, 410]), array([...",0,0.178919,0.343283,0.029334,0.048956,...,0.050168,0.005695,0.311323,0.038001,0.002517,0.000032,-1.760723,0.555551,2.644537,0.512206


In [5]:
df.columns

Index(['timestamp', 'path_img', 'ear', 'mar', 'landmarks', 'prediction',
       'ear_min', 'ear_max', 'mar_min', 'mar_max', 'ear_mean', 'mar_mean',
       'ear_std', 'mar_std', 'ear_median', 'mar_median', 'ear_var', 'mar_var',
       'ear_skew', 'mar_skew', 'ear_kurt', 'mar_kurt'],
      dtype='object')

In [30]:
num_landmarks = 4
X_train, X_temp, y_train, y_temp = train_test_split(df[["ear","mar", 'ear_median', 'mar_median']], df["prediction"], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [31]:
# Define the neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(num_landmarks,), name='input_layer'),  # Adjust num_landmarks to match your data
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [32]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
y_train=y_train.apply(lambda x : 1 if x == "fatiga" else 0)
y_val = y_val.apply(lambda x : 1 if x == "fatiga" else 0)
y_test = y_test.apply(lambda x : 1 if x == "fatiga" else 0)
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [34]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_val,y_val))

Epoch 1/20
8/8 [==============================] - 1s 22ms/step - loss: 0.6878 - accuracy: 0.7434 - val_loss: 0.6475 - val_accuracy: 1.0000
Epoch 2/20
8/8 [==============================] - 0s 5ms/step - loss: 0.6270 - accuracy: 1.0000 - val_loss: 0.5984 - val_accuracy: 1.0000
Epoch 3/20
8/8 [==============================] - 0s 5ms/step - loss: 0.5669 - accuracy: 1.0000 - val_loss: 0.5391 - val_accuracy: 1.0000
Epoch 4/20
8/8 [==============================] - 0s 5ms/step - loss: 0.4978 - accuracy: 1.0000 - val_loss: 0.4684 - val_accuracy: 1.0000
Epoch 5/20
8/8 [==============================] - 0s 6ms/step - loss: 0.4203 - accuracy: 1.0000 - val_loss: 0.3907 - val_accuracy: 1.0000
Epoch 6/20
8/8 [==============================] - 0s 5ms/step - loss: 0.3381 - accuracy: 1.0000 - val_loss: 0.3104 - val_accuracy: 1.0000
Epoch 7/20
8/8 [==============================] - 0s 6ms/step - loss: 0.2585 - accuracy: 1.0000 - val_loss: 0.2345 - val_accuracy: 1.0000
Epoch 8/20
8/8 [=================

In [35]:
X_test[0]

array([0.20656789, 0.04455394, 0.25099313, 0.03364868])

In [36]:


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Save the model for future use
model.save('fatigue_detection_model.h5')


2/2 [==============================] - 0s 4ms/step - loss: 0.0071 - accuracy: 1.0000
Test accuracy: 100.00%


/Users/rguarise/.pyenv/versions/3.11.0/envs/PFE/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('../models/fatigue_detection_model.h5')


1/1 [==============================] - 0s 52ms/step


array([[0.944017]], dtype=float32)

In [55]:
model.predict(X_test[1].reshape(1,2))

1/1 [==============================] - 0s 28ms/step


array([[0.13814627]], dtype=float32)